In [1]:
import sqlite3
import pandas as pd
import time
pd.set_option('display.width', 500)

In [3]:
inat_con = sqlite3.connect("inaturalist-open-data-20220127.sq3db")
print(inat_con.total_changes)

0


In [ ]:
start_time = time.time()
insect_url_df = pd.read_sql_query(
    "WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo_url) "
    "SELECT B.*, U.photo_url || A.photo_id || '/' || 'large.' || A.extension AS photo_url_large, D.name AS taxon_name, D.ancestry, D.taxon_id, A.photo_id, A.photo_uuid, A.extension "
    "FROM urls U "
    "CROSS JOIN photos A "
    "JOIN observers B "
    "ON A.observer_id = B.observer_id "
    "JOIN observations C "
    "ON A.observation_uuid = C.observation_uuid "
    "LEFT JOIN taxa D "
    "ON C.taxon_id = D.taxon_id "
    "WHERE D.ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND D.rank = 'species' "
    "AND C.quality_grade = 'research' ", 
    inat_con
)

print(insect_url_df.head())
insect_url_df.to_csv('iNaturalist_Insects_URLs.csv', index=False)
inat_con.close()
print("Executed in %.2f seconds" % (time.time() - start_time))

In [4]:
start_time = time.time()
insect_url_df = pd.read_csv('iNaturalist_Insects_URLs.csv')
print("Executed in %.2f seconds" % (time.time() - start_time))

In [5]:
start_time = time.time()
insect_con = sqlite3.connect("indexd-insects-inaturalist-open-data-20220127.sq3db")
insect_url_df.to_sql("insects", insect_con, if_exists="replace")
print("Executed in %.2f seconds" % (time.time() - start_time))

11516311

In [8]:
start_time = time.time()
cursor = insect_con.cursor()
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_ancestry" ON "insects" ("ancestry")')
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_login" ON "insects" ("login")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_name" ON "insects" ("name")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_observer_id" ON "insects" ("observer_id")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_photo_id" ON "insects" ("photo_id")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_photo_url_large" ON "insects" ("photo_url_large")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_photo_uuid" ON "insects" ("photo_uuid")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "idx_insects_taxon_name" ON "insects" ("taxon_name")') 
cursor.execute('CREATE UNIQUE INDEX IF NOT EXISTS "idx_observations_index" ON "insects" ("index")') 
cursor.execute('CREATE INDEX IF NOT EXISTS "ix_insects_index"ON "insects" ("index")') 

insect_con.commit()
print("Executed in %.2f seconds" % (time.time() - start_time))

Executed in 0.01 seconds


In [9]:
insect_con.close()